In [1]:
import pandas as pd
import pymc3 as pm
import numpy as np
import theano
import theano.tensor as tt

In [82]:
data = pd.read_csv("HerrmannThoeniGaechterDATA.csv", skiprows=3)

In [3]:
len(data)

65880

In [83]:
data.head()

,sessionid,groupid,mgroupid,subjectid,p,secseq,city,period,senderscontribution,otherscontribution,...,civic,ruleoflaw,female,age,numknown,singlechild,ageu21,urbanbackground,middleclass,membership
0,011108KA,703,703,701,N-experiment,0,Samara,1,20,18,...,8.050114,-0.884283,1,22.0,16.0,0.0,0,1.0,0.0,0.0
1,011108KA,703,703,701,N-experiment,0,Samara,1,20,10,...,8.050114,-0.884283,1,22.0,16.0,0.0,0,1.0,0.0,0.0
2,011108KA,703,703,701,N-experiment,0,Samara,1,20,10,...,8.050114,-0.884283,1,22.0,16.0,0.0,0,1.0,0.0,0.0
3,011108KA,702,702,702,N-experiment,0,Samara,1,10,10,...,8.050114,-0.884283,1,21.0,10.0,0.0,0,1.0,1.0,1.0
4,011108KA,702,702,702,N-experiment,0,Samara,1,10,20,...,8.050114,-0.884283,1,21.0,10.0,0.0,0,1.0,1.0,1.0


In [4]:
## constants
groupSize = 4
ntrials = 10
pi = 1.4
ntokens = 20
vals = list(range(21)) #possible values to contribute - from 0 to 20 tokens

## Prelim checks

In [5]:
len(set(data["subjectid"])) #should be 1120

1120

In [6]:
len(set(data["groupid"]))

280

In [7]:
data.dtypes

sessionid               object
groupid                  int64
mgroupid                 int64
subjectid                int64
p                       object
secseq                   int64
city                    object
period                   int64
senderscontribution      int64
otherscontribution       int64
punishment             float64
recpun                 float64
civic                  float64
ruleoflaw              float64
female                   int64
age                    float64
numknown               float64
singlechild            float64
ageu21                   int64
urbanbackground        float64
middleclass            float64
membership             float64
dtype: object

In [8]:
def get_gini(city):
    if city == "Melbourne":
        return 34.3
    if city == "Minsk":
        return 25.3
    if city == "Chengdu":
        return 38.5
    if city == "Copenhagen":
        return 28.7
    if city == "Bonn":
        return 31.9
    if city == "Athens":
        return 34.4
    if city == "Seoul":
        return 31.6
    if city == "Samara":
        return 37.5
    if city == "Zurich":
        return 32.7
    if city == "St. Gallen":
        return 32.7
    if city == "Istanbul":
        return 41.9
    if city == "Nottingham":
        return 34.8
    if city == "Dnipropetrovs'k":
        return 26.1
    if city == "Boston":
        return 41.1
    
data["gini"] = data["city"].apply(lambda x: get_gini(x))

In [9]:
#data = data[data["gini"].isna() == False]
#data = data.dropna()

In [10]:
len(data)

65880

In [11]:
redDat = data.iloc[::3, :] #get only every third row, the others are other responses that we don't need

In [12]:
redDat.reset_index(inplace = True, drop = True)

In [13]:
group_names = set(redDat["groupid"])
group_names = sorted(list(group_names))
ngroups = len(group_names)

### DOES SOMETHING FUCKED HERE

subject_names = set(redDat["subjectid"])
nsubjects = len(subject_names)
ngroups = 269

In [14]:
## No punish initialized
c_no_punish = np.zeros(shape = (groupSize, ntrials, ngroups))
Ga_no_punish = np.zeros(shape = (ntrials, ngroups))
Gc_no_punish = np.zeros(shape = (groupSize, ntrials, ngroups))

#punished initialized
c_punish = np.zeros(shape = (groupSize, ntrials, ngroups))
Ga_punish = np.zeros(shape = (ntrials, ngroups))
Gc_punish = np.zeros(shape = (groupSize, ntrials, ngroups))

#missing vector
missing = np.zeros(shape = (ngroups))

In [15]:
for g in range(ngroups):
    ### fancy way of fixing missing data - they will just become 0, 
    # and their position is then logged in "missing"-vector
    try: 
        #no punish
        c_no_punish[:,:,g][0] = redDat[(redDat["groupid"] == group_names[g]) & (redDat["p"] == "N-experiment")]["senderscontribution"][0:10].values
        c_no_punish[:,:,g][1] = redDat[(redDat["groupid"] == group_names[g]) & (redDat["p"] == "N-experiment")]["senderscontribution"][10:20].values
        c_no_punish[:,:,g][2] = redDat[(redDat["groupid"] == group_names[g]) & (redDat["p"] == "N-experiment")]["senderscontribution"][20:30].values
        c_no_punish[:,:,g][3] = redDat[(redDat["groupid"] == group_names[g]) & (redDat["p"] == "N-experiment")]["senderscontribution"][30:40].values
        
        #punish
        c_punish[:,:,g][0] = redDat[(redDat["groupid"] == group_names[g]) & (redDat["p"] == "P-experiment")]["senderscontribution"][0:10].values
        c_punish[:,:,g][1] = redDat[(redDat["groupid"] == group_names[g]) & (redDat["p"] == "P-experiment")]["senderscontribution"][10:20].values
        c_punish[:,:,g][2] = redDat[(redDat["groupid"] == group_names[g]) & (redDat["p"] == "P-experiment")]["senderscontribution"][20:30].values
        c_punish[:,:,g][3] = redDat[(redDat["groupid"] == group_names[g]) & (redDat["p"] == "P-experiment")]["senderscontribution"][30:40].values
    
    except: #could be more general... 
        missing[g] = 1
    
    #make "Ga"
    
    Ga_no_punish[:,g] = c_no_punish[:,:,g].mean(axis=0)
    Ga_punish[:,g] = c_punish[:,:,g].mean(axis=0)
    
    #make "Gc"
    for s in range(groupSize):
        Gc_no_punish[:,:,g][s] = np.delete(c_no_punish[:,:,g], s, 0).sum(axis=0) #just gives a single number though...
        Gc_punish[:,:,g][s] = np.delete(c_punish[:,:,g], s, 0).sum(axis=0)

### Concat the different conditions

In [16]:
c = np.zeros(shape = (groupSize, ntrials, ngroups, 2))
Ga = np.zeros(shape = (ntrials, ngroups, 2))

In [17]:
c.shape

(4, 10, 269, 2)

In [18]:
c[:,:,:,0] = c_no_punish
c[:,:,:,1] = c_punish

Ga[:,:,0] = Ga_no_punish
Ga[:,:,1] = Ga_punish

### Gini Coefficient

In [19]:
Gini = np.zeros(shape = ngroups)
for g in range(ngroups):
    Gini[g] = redDat[(redDat["groupid"] == group_names[g]) & (redDat["p"] == "P-experiment")]["gini"].values.mean()

In [20]:
mask = np.isnan(Gini) == False

In [21]:
Ga_punish = Ga_punish[:,mask]
Ga_no_punish = Ga_no_punish[:,mask]

c = c[:,:,mask,:]
Ga = Ga[:,mask,:]
#Gc = Gc[:,:,mask,:]
Gini = Gini[mask]

#get new ngroups
ngroups = len(Gini)

### Prelim check

First we need to add all contributions in a group and multiply them by 1.4 and calculate the Pearson correlation between Gini coefficients and the earnings.


### Heat map

Lower Quartile (Gini < 31.6)
Upper Quartile (Gini > 37.5)

## Decay model

$$c_{g,s,t}^\mu = c^0_{g,s} \cdot e^{-\gamma_{s,g} \cdot t}$$

$$c^0_{g,s} = \beta^c_0 + \beta^c_{Gini} \cdot Gini_g$$

$$\gamma_{s,g} = \beta^\gamma_0 + \beta^\gamma_{Gini} \cdot Gini_g$$

In [58]:
y = c[:,:,:,0]

In [59]:
idx_g = np.arange(ngroups)
idx_s = np.array([0,1,2,3])
t = np.arange(ntrials)

In [60]:
t_stack = np.stack((t,t,t,t))
t_stack = t_stack.T


In [81]:
Gini_stack

array([[32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [37.5, 37.5, 37.5, 37.5],
       [37.5, 37.5, 37.5, 37.5],
       [37.5, 37.5, 37.5, 37.5],
       [37.5, 37.5, 37.5, 37.5],
       [37.5, 37.5, 37.5, 37.5],
       [37.5, 37.5, 37.5, 37.5],
       [37.5, 37.5, 37.5, 37.5],
       [37

In [61]:
Gini_stack = np.stack((Gini, Gini, Gini, Gini))
Gini_stack = Gini_stack.T

In [62]:
y.shape

(4, 10, 244)

In [63]:
Gini_stack.shape

(244, 4)

In [64]:
idx_s = idx_s[:, None]
#idx_g = idx_g[:,None]

In [65]:
idx_s

array([[0],
       [1],
       [2],
       [3]])

In [66]:
idx_g

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [67]:
Gini_stack[idx_g, idx_s]

array([[32.7, 32.7, 32.7, 32.7, 32.7, 32.7, 32.7, 32.7, 32.7, 32.7, 32.7,
        32.7, 32.7, 32.7, 32.7, 32.7, 32.7, 32.7, 32.7, 32.7, 32.7, 32.7,
        32.7, 37.5, 37.5, 37.5, 37.5, 37.5, 37.5, 37.5, 37.5, 37.5, 37.5,
        37.5, 37.5, 37.5, 37.5, 37.5, 37.5, 37.5, 37.5, 37.5, 37.5, 37.5,
        37.5, 37.5, 37.5, 37.5, 37.5, 37.5, 37.5, 37.5, 37.5, 37.5, 37.5,
        37.5, 37.5, 37.5, 37.5, 37.5, 37.5, 25.3, 25.3, 25.3, 25.3, 25.3,
        25.3, 25.3, 25.3, 25.3, 25.3, 25.3, 25.3, 25.3, 25.3, 25.3, 25.3,
        25.3, 41.1, 41.1, 41.1, 41.1, 41.1, 41.1, 41.1, 41.1, 41.1, 41.1,
        41.1, 41.1, 41.1, 41.1, 32.7, 32.7, 32.7, 32.7, 32.7, 32.7, 32.7,
        32.7, 32.7, 32.7, 32.7, 32.7, 28.7, 28.7, 28.7, 28.7, 28.7, 34.8,
        34.8, 34.8, 34.8, 34.8, 34.8, 32.7, 32.7, 32.7, 32.7, 32.7, 32.7,
        32.7, 32.7, 32.7, 32.7, 32.7, 32.7, 28.7, 28.7, 26.1, 26.1, 26.1,
        26.1, 26.1, 26.1, 26.1, 26.1, 26.1, 26.1, 26.1, 34.8, 34.8, 34.8,
        34.8, 34.8, 34.8, 34.8, 34.8, 

In [76]:
Gini_stack

array([[32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [32.7, 32.7, 32.7, 32.7],
       [37.5, 37.5, 37.5, 37.5],
       [37.5, 37.5, 37.5, 37.5],
       [37.5, 37.5, 37.5, 37.5],
       [37.5, 37.5, 37.5, 37.5],
       [37.5, 37.5, 37.5, 37.5],
       [37.5, 37.5, 37.5, 37.5],
       [37.5, 37.5, 37.5, 37.5],
       [37

In [73]:
with pm.Model() as m:
    ## priors
    beta0_c0 = pm.HalfNormal("beta0_c0", 1)
    betaGini_c0 = pm.Normal("betaGini_c0", 0, 1)
    
    beta0_gamma = pm.HalfNormal("beta0_gamma", 1)
    betaGini_gamma = pm.Normal("betaGini_gamma", 0, 1)
    
    ## group_level
    sigma_c = pm.Gamma("sigma_c", 1, 1, shape = (ngroups, groupSize))
    
    c_0 = pm.Deterministic("c_0", beta0_c0 + betaGini_c0 * Gini_stack[idx_g, idx_s])
    
    gamma = pm.Deterministic("gamma", beta0_gamma + betaGini_gamma * Gini_stack[idx_g, idx_s])
    
    ### with time 
    
    tester = -gamma * t_stack
    
    test_2 = pm.math.dot(tester, c_0)
    
    mu_c = pm.Deterministic("mu_c", 
                            c_0 * pm.math.exp(pm.math.dot(t_stack, -gamma))) ## this is gonna be fucked
    
    ## likelihood
    
    y_pred = pm.Normal("y_pred", mu = mu_c, sigma = sigma_c, observed = y)
    
    

ValueError: Input dimension mis-match. (input[0].shape[0] = 4, input[1].shape[0] = 10)

### Preferences:

$$pvec = p_0 + \beta_p \cdot tokenvalues $$

$$p_t = pvec[Gb_t]$$

where Gb_t is the belief of what the other person will do at time T

$$Gb_t = \gamma ( Gb_{t-1}) + (1-\gamma) (G_{a_{t-1}})$$

$$c_t = \omega_t (Gb_t) + (1-\omega_t) (p_t)$$

$$\omega_t = \omega_{t-1}(1-\lambda)$$

### Getting the hierachical model

$$B^P_{g,s} \sim Beta(Shape_{g,s,1}, Shape_{g,s,2})$$

$$Shape_1 = \mu B^P_{g,s} + \sigma_{g,s}$$
$$Shape_2 = (1 - \mu B^P_{g,s}) + \sigma_{g,s}$$

$$Probit(\mu B^P_{g,s}) = \beta_0^P + \beta^P_{Gini} \cdot Gini$$

In [40]:
t

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])